# News Clustering using KMeans Algorithm
By Datetime : 2016-08-29 ~ 2016-09-05

In [1]:
import numpy as np
import pandas as pd

## Load data from Pickle 

In [2]:
train = pd.read_pickle("../datastore/1104_00_df.p")
category = pd.read_pickle("../datastore/category.p")
category.head()

,category,name
0,건강·의학 >,건강
1,경제,경제
2,뉴스 > 경제,경제
3,경제 >,경제
4,뉴스 > 경제 > @뉴스룸,경제


In [3]:
headline = pd.read_pickle("../datastore/headline.p")
headlines = headline['headline'].tolist()

In [4]:
headlines

['지표로 보는 경제',
 '오늘의',
 '화제의 분양현장',
 '뉴스룸',
 '카드뉴스',
 '조선일보',
 'TV조선',
 '특집',
 '이슈',
 'Weekly BIZ',
 '금주의',
 '사회공헌 Together',
 '신나는 공부',
 '포토',
 '리빙포인트',
 '경제계 인사',
 '경향포토',
 '오늘의 날씨']

## Select Categories
- 포함 : 경제, 문화, 건강, 과학, 사회, 정치, 스포츠
- 제외 : 종합, 정보없음, 인물, 사설

In [5]:
train = pd.merge(train, category, on='category', how='inner')

In [6]:
categories = ['경제', '과학', '사회', '정치']

In [7]:
train = train[train['name'].isin(categories)]
train.name.unique()

array(['사회', '정치', '경제', '과학'], dtype=object)

## Preprocessing
1. Datetime (16-09-11 ~ 16-09-17)
2. Remove stopwords (regex, hanja)
3. POS Tagging with KoNLPy, Mecab
4. Using bigram

In [8]:
import datetime
from konlpy.tag import Mecab
import hanja
import re

In [9]:
mecab = Mecab()

In [10]:
def text_cleaning(text):
    text = hanja.translate(text, 'substitution')
    text = re.sub('[^가-힝0-9a-zA-Z\\s]', ' ', text)
    for headline in headlines:
        text = text.replace(headline, ' ')
    return text

In [11]:
def tokenize(data):
    return [' '.join(e for e in mecab.nouns(data))]

In [12]:
train['title_flat'] = train['title'].apply(lambda text: text_cleaning(text))
title = [tokenize(each[1]['title_flat']) for each in train.iterrows()]

## Training
1. Feature extraction - TfidVectorizer
2. Decomposition - PCA
3. Cluster - KMeans

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [14]:
vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,2))
title_flat = [item for sublist in title for item in sublist]
x_list = vectorizer.fit_transform(title_flat)

In [15]:
x_list_100d = PCA(n_components=100).fit_transform(x_list.toarray())
x_list_100d.shape

(2802, 100)

### Scoring

In [16]:
from sklearn.metrics import silhouette_samples, silhouette_score
from IPython.display import display, HTML
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### Best Silhoutte Score

In [17]:
best_score = 0.0
best_k = 0

In [18]:
for k in range(15, 35):
    km = KMeans(n_clusters=k, n_jobs=-1).fit(x_list_100d)
    score = silhouette_score(x_list_100d, km.labels_)
    if best_score < score:
        best_score = score
        best_k = k
    print("In Clusters =", k, ", Score is : %0.3f" % score)
print("In Clusters =", best_k, ", Best score is : %0.3f" % best_score)

In Clusters = 15 , Score is : 0.227
In Clusters = 16 , Score is : 0.158
In Clusters = 17 , Score is : 0.054
In Clusters = 18 , Score is : 0.194
In Clusters = 19 , Score is : 0.151
In Clusters = 20 , Score is : 0.171
In Clusters = 21 , Score is : 0.205
In Clusters = 22 , Score is : 0.149
In Clusters = 23 , Score is : 0.234
In Clusters = 24 , Score is : 0.185
In Clusters = 25 , Score is : 0.203
In Clusters = 26 , Score is : 0.125
In Clusters = 27 , Score is : 0.137
In Clusters = 28 , Score is : 0.209
In Clusters = 29 , Score is : 0.222
In Clusters = 30 , Score is : 0.124
In Clusters = 31 , Score is : 0.162
In Clusters = 32 , Score is : 0.190
In Clusters = 33 , Score is : 0.182
In Clusters = 34 , Score is : 0.199
In Clusters = 23 , Best score is : 0.234


### K-Means Algorithm

In [19]:
km = KMeans(n_clusters=best_k, n_jobs=-1).fit(x_list_100d)
labels = km.labels_
centroids = km.cluster_centers_
print(km.inertia_)

301.838414853


In [20]:
x_list_vector = x_list_100d.tolist()
train = train.drop(['title_flat', 'target_str'], axis=1)
train['cluster'] = labels

## Choose Best Cluster
1. Cluster size < 500
2. Recent published
3. Minimum inertia

### Compare best cluster

In [21]:
sample_silhouette_values = silhouette_samples(x_list_100d, labels)
sample_silhouette_score = []
best_cluster = []
cluster_num = best_k

for i in range(cluster_num):
    ith_cluster_silhouette_values = \
        sample_silhouette_values[labels == i]
        
    print('Cluster %d: ' % (i), abs(ith_cluster_silhouette_values.mean()))
    sample_silhouette_score.append(abs(ith_cluster_silhouette_values.mean()))

sample_silhouette_score.sort(reverse=True)
sample_silhouette_score = sample_silhouette_score[:8]

Cluster 0:  0.0502198592067
Cluster 1:  0.258042921253
Cluster 2:  0.379140482439
Cluster 3:  0.760952191998
Cluster 4:  0.837904819713
Cluster 5:  0.216879778453
Cluster 6:  0.353200095946
Cluster 7:  0.0554535790352
Cluster 8:  0.167773083158
Cluster 9:  0.0999849017844
Cluster 10:  0.312123250329
Cluster 11:  0.103006433494
Cluster 12:  0.999999995599
Cluster 13:  0.106193143989
Cluster 14:  0.248349660093
Cluster 15:  0.118036434433
Cluster 16:  0.135426041306
Cluster 17:  0.749364540736
Cluster 18:  0.427068973283
Cluster 19:  0.0679168919714
Cluster 20:  0.459287525144
Cluster 21:  0.905722600522
Cluster 22:  0.339418177589


In [22]:
sample_silhouette_score

[0.99999999559937858,
 0.90572260052166753,
 0.83790481971297182,
 0.76095219199823039,
 0.74936454073619818,
 0.45928752514385568,
 0.42706897328320004,
 0.37914048243912185]

In [23]:
for i in range(cluster_num):
    ith_cluster_silhouette_values = \
        sample_silhouette_values[labels == i]
        
    if abs(ith_cluster_silhouette_values.mean()) in sample_silhouette_score:
        best_cluster.append(i)

In [24]:
best_cluster

[2, 3, 4, 12, 17, 18, 20, 21]

In [25]:
train = train[train['cluster'].isin(best_cluster)]
train.cluster.unique()

array([18,  2, 20, 21, 17, 12,  3,  4])

## Result

In [26]:
cluster_data = []

for cluster_index in range(cluster_num):
    if cluster_index in best_cluster:
        cluster_data.append(train[train['cluster'] == cluster_index])
    
for i, d in enumerate(cluster_data):
    print('Cluster %d:' % (i), 'Size %d' % (len(d)))

    display(d[['title', 'category']].sample(min(10, len(d))))
    print('\n\n')

Cluster 0: Size 24


,title,category
296,[경향포토]운영위 답변하는 한광옥 비서실장,정치 > 정치일반
291,[경향포토]국회 더불어민주당 찾은 비서실장,정치 > 정치일반
299,[경향포토]국회 운영위 나온 한광옥 대통령 비서실장,정치 > 정치일반
211,[11·2 기습 개각]“비서실장 누가 맡으려 할까” 뒤엉켜버린 인적쇄신 수순,정치 > 정치일반
289,[경향포토]국회 국민의당 찾은 한광옥 비서실장,정치 > 정치일반
290,[경향포토]국회 국민의당 찾은 비서실장,정치 > 정치일반
298,[경향포토]국회 운영위 답변하는 한광옥 대통령 비서실장,정치 > 정치일반
1649,15년 만에 다시 비서실장 기용된 한광옥,뉴스 > 정치 > 정치일반
292,[경향포토]국회 온 비서실장,정치 > 정치일반
293,[경향포토]국회 새누리당 찾은 한광옥 비서실장,정치 > 정치일반





Cluster 1: Size 6


,title,category
2010,"서울빛초롱축제, 4일 청계천서 개막",뉴스 > 사회 > 사회일반
708,[경향포토]서울빛초롱축제 내일 개막,사회 > 사회일반
711,[경향포토]청계천 '2016 서울빛초롱축제',사회 > 사회일반
709,[경향포토]2016 서울 빛초롱 축제,사회 > 사회일반
2323,서울 청계천서 오늘부터 ‘빛초롱축제’ 연다,전국 > 서울·수도권 서울 경기 인천
710,[경향포토]청계천 서울빛초롱축제,사회 > 사회일반





Cluster 2: Size 7


...

,title,category
81,민주당 추미애 “법적 권한 없는 거국중립내각은 장식용 내각 불과”,정치 > 정치일반
903,거국중립내각 먼저 꺼내고도 머리 아픈 야당,정치 > 국회·정당
937,‘거국중립내각’ 왜 말만 요란할까?,정치 > 국회·정당
905,"박대통령, 우병우·안종범·‘3인방’ 경질 새누리, 거국중립내각 구성 요구",정치 > 국회·정당
80,"與 정진석, 거국중립내각 거부한 야권에 “탄핵정국, 하야정국 몰고가려 하나""",정치 > 정치일반
104,[최순실 국정농단]여·야 ‘거국중립내각’ 같은 말 다른 속내,정치 > 정치일반
1538,"與, 거국중립내각 구성 요구",뉴스 > 정치 > 정치일반
893,‘거국중립내각’ 요구 분출…여 잠룡들도 가세,정치 > 국회·정당
52,새누리 “거국중립내각 구성해야”,정치 > 정치일반
1194,"청, 거국중립내각 반대…이원종 실장 26일 사표",정치 > 청와대





Cluster 6: Size 7


,title,category
196,[경향포토]야3당 원내대표 회의,정치 > 정치일반
131,[경향포토]손잡은 야3당 원내대표,정치 > 정치일반
195,[경향포토]야3당 원내대표 회동,정치 > 정치일반
136,[경향포토]최순실 관련 야3당 원내대표 회동,정치 > 정치일반
137,[경향포토]야3당 원내대표 회동,정치 > 정치일반
83,[속보]국회의장·여야 원내대표 회동 10분 만에 결렬,정치 > 정치일반
2767,10분만에 깨진 3黨 원내대표 회동,정치 > 국회ㆍ정당





Cluster 7: Size 3


,title,category
1576,朴대통령 “제가 邪敎(사교)에 빠졌다니… ”,뉴스 > 정치 > 정치일반
158,박대통령 “저더러 사교(邪敎)에 빠졌다고 하더라구요”,정치 > 정치일반
150,박 대통령 “저더러 ‘사교’에 빠졌다고 하더라구요”,정치 > 정치일반


## Save Dataframe to MongoDB

In [27]:
from pymongo import MongoClient

In [28]:
client = MongoClient('mongodb://localhost:27017/somanews')
client.somanews.authenticate('ssomanews', 'ssomanews1029')
db = client.get_database('somanews')
articles = db.get_collection('articles')

In [29]:
articles.insert_many(train.to_dict(orient='records'))
client.close()